In [14]:
import numpy as np
import pandas as pd 
from DataLoader import *

In [15]:
data_path = "../../WHAI-provided_data/"

In [16]:
p_path = data_path+"02_picking-activity_K1.csv"
i_path = data_path+"04_Item-Master_K1.xlsx"
clayout_path = data_path+"Current_layout.xlsx"


def preprocessing(picking_path=p_path):
    picking = pd.read_csv(picking_path)
    picking["Execution_time"] = pd.to_datetime(picking["Execution_time"])
    
    Item = pd.read_excel(i_path)
    Item = Item[~Item["Article number"].isna()]
    Item["Article number"] = Item["Article number"].astype(int)
    
    picking = picking.merge(Item[["Article number","number per load carrier"]],
                  left_on = "Artikelno",right_on = "Article number", how="left")
    del picking['Article number']
    
    return picking

In [83]:
picking = preprocessing(p_path)

In [90]:
picking = picking.rename(columns={"number per load carrier": "Unit_load"})
picking

,Execution_time,Amount,Order,Artikelno,Charge,Run,From_pallet,Origin,Destination,Date_receipt,Date_MHD,Recipient,Unit_load
0,2018-08-31,1,983500824,398,28022021,180903,114348,2-08-122-1,40,8/31/18,2/28/21,E500,32
1,2018-08-31,1,983500824,251,10072019,180903,114284,2-05-121-1,40,8/31/18,7/10/19,E500,162
2,2018-08-31,2,983500824,127,9012020,180903,114252,2-03-119-1,40,8/31/18,1/9/20,E500,114
3,2018-08-31,1,983500824,160,25072019,180903,114246,2-03-107-1,40,8/31/18,7/25/19,E500,55
4,2018-08-31,1,983500824,370,5072019,180903,114218,2-02-118-1,40,8/31/18,7/5/19,E500,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311499,2020-07-31,27,543711,177,6012022,200731,891387,2-30-143-1,44,7/8/20,1/6/22,E33,210
311500,2020-07-31,8,543711,38,9062021,200731,861209,2-29-132-1,44,6/10/20,6/9/21,E33,162
311501,2020-07-31,17,543711,190,19122021,200731,872294,2-31-136-1,44,6/19/20,12/19/21,E33,133
311502,2020-07-31,21,543711,149,18042021,200731,871883,2-27-130-1,44,6/19/20,4/18/21,E33,100


In [95]:
model_input = picking[["Execution_time","Artikelno","Amount","Unit_load"]].groupby(
    ["Artikelno","Unit_load"],as_index = 0)["Amount"].agg({"Total":'sum',"Picks":'count'}).reset_index(drop=True)
model_input

,Artikelno,Unit_load,Total,Picks
0,1,57,1448,621
1,2,200,526,12
2,3,420,32564,1620
3,4,40,582,81
4,5,182,2379,101
...,...,...,...,...
615,1949,165,141,5
616,1950,165,141,5
617,1951,165,141,5
618,1952,165,207,5


In [96]:
model_input["Full_pallet"] = np.ceil(model_input["Total"]/model_input["Unit_load"]).astype(int)
model_input

,Artikelno,Unit_load,Total,Picks,Full_pallet
0,1,57,1448,621,26
1,2,200,526,12,3
2,3,420,32564,1620,78
3,4,40,582,81,15
4,5,182,2379,101,14
...,...,...,...,...,...
615,1949,165,141,5,1
616,1950,165,141,5,1
617,1951,165,141,5,1
618,1952,165,207,5,2


In [97]:
Clayout= read_clayout(clayout_path)
unique, counts = np.unique(Clayout.to_numpy().flatten(), return_counts=True)
li = pd.DataFrame({"Artikelno":unique[2:],"Min_area":counts[2:],"Assigned":np.zeros_like(counts[2:])})
model_input = model_input.merge(li,on="Artikelno",how="left")
model_input["Min_area"] = model_input["Min_area"].fillna(1).astype(int)
model_input["Assigned"] = model_input["Assigned"].fillna(0).astype(int)
model_input

,Artikelno,Unit_load,Total,Picks,Full_pallet,Min_area,Assigned
0,1,57,1448,621,26,1,0
1,2,200,526,12,3,1,0
2,3,420,32564,1620,78,1,0
3,4,40,582,81,15,1,0
4,5,182,2379,101,14,1,0
...,...,...,...,...,...,...,...
615,1949,165,141,5,1,1,0
616,1950,165,141,5,1,1,0
617,1951,165,141,5,1,1,0
618,1952,165,207,5,2,1,0


# Model

## parameters


In [54]:
A = model_input["Artikelno"].to_numpy()
pi = dict(zip(A,model_input["Picks"].to_numpy()))
di = dict(zip(A,model_input["Full_pallet"].to_numpy()))

In [55]:
li = dict(zip(A,model_input["min_area"].to_numpy()))

In [56]:
s = 2
cr = 2 

N = np.count_nonzero(Clayout.to_numpy().flatten()!=-1)

In [57]:
N

246

In [62]:
model = Model("")
x_i = {}
for i in A:
    x_i[i] = model.addVar(vtype=GRB.BINARY, name='x_i[%s]' %(i))
    
C1 = model.addConstr((quicksum(x_i[i]*li[i] for i in A) <= N), "")

model.setObjective(quicksum((s*pi[i]-cr*di[i])*x_i[i] for i in A), GRB.MAXIMIZE)
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1 rows, 620 columns and 620 nonzeros
Model fingerprint: 0x6be0d888
Variable types: 0 continuous, 620 integer (620 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+02]
Found heuristic solution: objective 488202.00000
Presolve removed 1 rows and 620 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 544390 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.443900000000e+05, best bound 5.443900000000e+05, gap 0.0000%


In [69]:
solution = {}
select = []
for v in model.getVars():
    solution[v.VarName[4:-1]] = v.x
    if v.x ==1 :
        select.append(int(v.VarName[4:-1]))
select

[1,
 3,
 6,
 8,
 9,
 10,
 11,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 26,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 69,
 70,
 71,
 72,
 73,
 76,
 77,
 78,
 79,
 80,
 83,
 86,
 87,
 88,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 136,
 137,
 139,
 140,
 141,
 142,
 144,
 146,
 147,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 184,
 186,
 187,
 188,
 189,
 190,
 193,
 194,
 195,
 196,
 197,
 198,
 201,
 204,
 205,
 207,
 208,
 212,
 213,
 214,
 218,
 219,
 220,
 222,
 224,
 225,
 226,
 227,
 228,
 230,
 231,
 232,
 233,
 235,
 236,
 2

In [74]:
I = set(unique[2:].tolist())
M = set(select)

216

In [81]:

len(M.intersection(I))

198